In [ ]:
from tqdm.auto import tqdm
import numpy as np
from collections import Counter

from datasets import load_dataset, Dataset

In [ ]:
oasst1_dataset = load_dataset('OpenAssistant/oasst1', split="train+validation")
lima_dataset = load_dataset('GAIR/lima', split="train")

In [ ]:
root_nodes = {}
for sample in tqdm(oasst1_dataset):
    if sample['parent_id'] is None:
        root_nodes[sample['message_id']] = sample
root_ids = set(root_nodes)

In [ ]:
block_words = ['open assist', 'open-assist', 'chatgpt', 'laion']

nice_data = []
for sample in tqdm(oasst1_dataset):
    if sample['lang'] == 'en' and sample['rank'] == 0 and sample['parent_id'] in root_ids:
        if any(block_word in sample['text'].lower() for block_word in block_words):
            continue

        if any(block_word in root_nodes[sample['parent_id']]['text'].lower() for block_word in block_words):
            continue

        nice_data.append([
            root_nodes[sample['parent_id']],
            sample
        ])
print(len(nice_data))

In [ ]:
# convs = [(sample[0]['text'], sample[1]['text']) for sample in nice_data]
convs = [[sample[0]['text'], sample[1]['text']] for sample in nice_data]

lima_convs = [[a, b] for a, b in lima_dataset[:1000]['conversations']]

In [ ]:
all_conversations = convs + lima_convs
all_source = ['oasst']*len(convs) + ['lima'] * 1000

In [ ]:
len(all_conversations), len(all_source)

In [ ]:
msg_ds = Dataset.from_dict({"conversations": all_conversations, "source": all_source})

In [ ]:
msg_ds

In [ ]:
msg_ds[-10]

In [ ]:
msg_ds.push_to_hub("ycchen_submission_2_dataset")